In [17]:
import os
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer
from tqdm.auto import tqdm, trange

plt.style.use('fivethirtyeight')

EOS = '<|endoftext|>'
POSITIVE_TOKEN = "<|positive|>"
NEGATIVE_TOKEN = "<|negative|>"

In [18]:
model_path = '../deep-latent-sequence-model/yelp_gpt_aug/checkpoint-1260/'
assert os.path.exists(model_path)

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_tokens(["<|positive|>", "<|negative|>"])
gpt = GPT2LMHeadModel.from_pretrained(model_path).eval()

In [19]:
def sample_sentence_from(model, tokenizer, sentence, stop_after_dot=True):
    while not EOS in sentence and len(sentence) < 300:
        if stop_after_dot and '.' in sentence:
            break
        tokens = tokenizer(sentence)['input_ids']
        logits = model(torch.tensor(tokens))[0][-1].detach()
        weights = logits.softmax(0).detach().numpy()
        prediction = random.choices(list(range(len(logits))), weights=logits.softmax(0))[0]
        sentence += tokenizer.decode(prediction)
    return sentence

In [20]:
random.seed(1)
for _ in trange(30):
    print(sample_sentence_from(gpt, tokenizer, "<|positive|>", stop_after_dot=False))

<|positive|>mango juice is great. <|endoftext|>
<|positive|>i was impressed with their waffle keeper and cheesecake and the macaroons were amazing - they were worth the price. <|endoftext|>
<|positive|>: wonderful staff, great waiter staff and great customer service! <|endoftext|>
<|positive|>flavor point : lemon! ( <|endoftext|>
<|positive|>plus they have such great reviews there. <|endoftext|>
<|positive|>anyway, the only food asset was the service. Experiment if you want to use homemade dish. <|endoftext|>
<|positive|>bill_ '' helped. also service was fine. <|endoftext|>
<|positive|>the food was quite good most of the time. <|endoftext|>
<|positive|>taco bell pizza, pizza or sandwiches. <|endoftext|>
<|positive|>1 day or more then 5-6 weeks for my meal. <|endoftext|>
<|positive|>I 'm not a terrible customer but when i was a little kid we went there. <|endoftext|>
<|positive|>plus some great stops for adults and kids as well! Get in early and grab your carbonated bbq!<|endoftext|>
<|

In [21]:
random.seed(1)
for _ in trange(30):
    print(sample_sentence_from(gpt, tokenizer, "<|negative|>", stop_after_dot=False))

<|negative|>lisa, you 're perfect! <|endoftext|>
<|negative|>originally, izmas was folk midnight to dine probably. <|endoftext|>
<|negative|>I had the doctor who was very helpful and on top of everything. Highly recommended. <|endoftext|>
<|negative|>Donna was wonderful. <|endoftext|>
<|negative|>had no idea what a bit gem they were making. <|endoftext|>
<|negative|>perusal : good food, and a park vibe. <|endoftext|>
<|negative|>comparable price. i would recommend! <|endoftext|>
<|negative|>an excellent seafood omelette! <|endoftext|>
<|negative|>ful service at a reasonable price. <|endoftext|>
<|negative|>ersatz food. <|endoftext|>
<|negative|>I had a heart attack last night. <|endoftext|>
<|negative|> changes of the so called last year. <|endoftext|>
<|negative|>rude and mean, unprofessional. <|endoftext|>
<|negative|>'t this be funny? <|endoftext|>
<|negative|>even averted through the windows. <|endoftext|>
<|negative|>nice hotel. <|endoftext|>
<|negative|>elope was happy to say hel

Unfortunately, sentiment is not preserved when we use this way of training GPT. It seems, that it doesn't really udnerstand sentiment, and the fact that it is supposed to generate negative sentences after <|negative|> token and positive after <|positive|>.

```
1 <|positive|>mango juice is great. <|endoftext|>
1<|positive|>i was impressed with their waffle keeper and cheesecake and the     macaroons were amazing - they were worth the price. <|endoftext|>
1<|positive|>: wonderful staff, great waiter staff and great customer service! <|endoftext|>
0<|positive|>flavor point : lemon! ( <|endoftext|>
1<|positive|>plus they have such great reviews there. <|endoftext|>
0<|positive|>anyway, the only food asset was the service. Experiment if you want to use homemade dish. <|endoftext|>
1<|positive|>bill_ '' helped. also service was fine. <|endoftext|>
1<|positive|>the food was quite good most of the time. <|endoftext|>
0<|positive|>taco bell pizza, pizza or sandwiches. <|endoftext|>
0<|positive|>1 day or more then 5-6 weeks for my meal. <|endoftext|>
0<|positive|>I 'm not a terrible customer but when i was a little kid we went there. <|endoftext|>
1<|positive|>plus some great stops for adults and kids as well! Get in early and grab your carbonated bbq!<|endoftext|>
1<|positive|>the food was phenomenal, with other parts of the movie. <|endoftext|>
1<|positive|>entajos really stepped in in and tasted excellent. <|endoftext|>
0<|positive|>full glory after eating here. tasted like plain. <|endoftext|>
-1<|positive|>in response to another user 's reply, stated `terrible rep tl ;dr `.. Also, did n't know why their fries were so bland. <|endoftext|>
-1<|positive|>i can barely get in· they are so slow. <|endoftext|>
0<|positive|>eliza grew up in yelp and sesame season. <|endoftext|>
0<|positive|>dessert meanwhile! <|endoftext|>
1<|positive|>of immaculate taste and cleanliness. <|endoftext|>
1<|positive|>rats. btw nice community if you like fancy fried.......and a great, warm one for travel. $ _num_.

1<|positive|>extremely fast service! _num_ minutes wait for professionals to arrive in. Food, drinks, prices, and environment are great. <|endoftext|>
1<|positive|>Override & spread pro recommended. <|endoftext|>
1<|positive|>an all around great experience for me and my family here. <|endoftext|>
0<|positive|> o- matian style small appetizers for $ _num_. <|endoftext|>
-1<|positive|> of course the guacamole there had to be smoked. <|endoftext|>
1<|positive|>to dogs! angel town is that good... whatever the situation, this should be good nuggets! <|endoftext|>
1<|positive|>got ta go here today. <|endoftext|>
0<|positive|>perfect gems... for the money!<|endoftext|>
1<|positive|>i got the entree and i was very pleased with it. <|endoftext|>
0<|positive|> and my heart goes out to every fan! <|endoftext|>
```

16 positive, 3 negative, 11 neutral / 30 positive generations

```
1<|negative|>lisa, you 're perfect! <|endoftext|>
0<|negative|>originally, izmas was folk midnight to dine probably. <|endoftext|>
1<|negative|>I had the doctor who was very helpful and on top of everything. Highly recommended. <|endoftext|>
1<|negative|>Donna was wonderful. <|endoftext|>
-1<|negative|>had no idea what a bit gem they were making. <|endoftext|>
1<|negative|>perusal : good food, and a park vibe. <|endoftext|>
1<|negative|>comparable price. i would recommend! <|endoftext|>
1<|negative|>an excellent seafood omelette! <|endoftext|>
1<|negative|>ful service at a reasonable price. <|endoftext|>
-1<|negative|>ersatz food. <|endoftext|>
-1<|negative|>I had a heart attack last night. <|endoftext|>
0<|negative|> changes of the so called last year. <|endoftext|>
-1<|negative|>rude and mean, unprofessional. <|endoftext|>
0<|negative|>'t this be funny? <|endoftext|>
0<|negative|>even averted through the windows. <|endoftext|>
1<|negative|>nice hotel. <|endoftext|>
0<|negative|>elope was happy to say hello, and understood me somewhat. <|endoftext|>
-1<|negative|>-Ok the salad dressing was not cooked correctly. <|endoftext|>
-1<|negative|>terrible! <|endoftext|>
1<|negative|>i went to her for lunch today and was really pleasantly surprised. <|endoftext|>
0<|negative|>owned i 've never been wed or ringer it yet. <|endoftext|>
1<|negative|>delicious. <|endoftext|>
1<|negative|> --boarding trip of course is a very pleasant experience. <|endoftext|>
-1<|negative|>poisonous. <|endoftext|>
1<|negative|>six star ratings. <|endoftext|>
-1<|negative|>first the lunch was soooo over difficult! <|endoftext|>
-1<|negative|>sadly we ordered the buffet sauce and it was not deep and sultry. <|endoftext|>
0<|negative|>crikulus wo n't be picking up again. <|endoftext|>
0<|negative|>as is the case throughout the whole animal realm. <|endoftext|>
1<|negative|>rosettlor was very efficient with our food and service. <|endoftext|>
```

13 positive, 9 negative, 9 neutral / 30 negarive generations

# The Big Question: How to incorporate Lanugage Understanding in Language Models?
It seems, that even the best models don't really have it. It is really sad. How do people understand language?